In [1]:
import collections
import copy
import datetime
import itertools
import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
G = cibblbibbl.CIBBL
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay

In [3]:
data = collections.defaultdict(list)
for Mu in G.matchups:
    for teamId, d0 in Mu.config["player"].items():
        for playerId, p in d0.items():
            Pl = player(str(playerId))
            prespp = p.get("prespp", 0)
            spp = p.get("spp", 0)
            data[Pl].append((Mu, prespp, spp))

In [8]:
for Pl in sorted(data):
    L = data[Pl]
    if not isinstance(Pl, (cibblbibbl.player.Player, cibblbibbl.player.RaisedDeadPlayer)):
        continue
    expprespp = 0
    if Pl.prev:
        if data.get(Pl.prev):
            prevlastd = data[Pl.prev][-1]
        else:
            prevlastd = [None, 0, 0]
        expprespp = sum(prevlastd[1:3]) * Pl.prevsppmul
    nameprint = False
    for i, (Mu, prespp, spp) in enumerate(L):
        if prespp != expprespp:
            if not nameprint:
                s = f'[{Pl.Id}, {Pl.status}] {Pl.name}'
                if Pl.prev:
                    s += f' (prev: [{Pl.prev.Id}] {Pl.prev.name})'
                print(s)
                nameprint = True
            print(f'{Mu.match.Id}, exp:{expprespp}, got:{prespp}')
        expprespp = prespp + spp       
    if nameprint:
        print()

[11744537, Active] Haseem Abdul Sabbar (prev: [11736871] Haseem Abdul Sabbar)
3897789, exp:5, got:0

[11765469, Dead] Fayid Mohammed (prev: [11736876] Fayid Mohammed)
3900603, exp:3, got:0

[11791157, Dead] Calas Typhon (prev: [11558607] Calas Typhon)
3905436, exp:10, got:0

[11940785, Retired] Varesh Kaurambi (prev: [11770088] Varesh Kaurambi)
3930641, exp:7, got:0

[11956674, Retired] Abd Allah Bousaid (prev: [11455996] Abd Allah Bousaid)
3933541, exp:33, got:0

[11978228, Retired] Oogie Boogie (prev: [11881505] Oogie Boogie)
3934598, exp:18, got:0

[12138861, Dead] Gaudi (prev: [11881471] Gaudi)
3964222, exp:32, got:0

[12349401, Retired] Kkrozh (prev: [12348495] tochtiguhtobias)
4046041, exp:5, got:8

[12372276, Active] Zzak (prev: [12295363] Zzak'tarh)
4010498, exp:14, got:0
4046041, exp:3, got:17

[12372278, Active] Tarh (prev: [12295363] Zzak'tarh)
4010498, exp:14, got:0
4046041, exp:9, got:23

[12423015, Active] Harkon Halvarson (prev: [11892408] Harkon Halvarson)
4017192, exp:

In [9]:
Pl0 = player("12349402")

In [10]:
data[Pl0]

[(Matchup('cibbl', '45237', 4, 888760, 908529), 0, 0),
 (Matchup('cibbl', '45577', 1, 907426, 908529), 0, 0),
 (Matchup('cibbl', '45577', 2, 908529, 910344), 0, 0),
 (Matchup('cibbl', '45577', 3, 908411, 908529), 0, 0)]

In [14]:
Pl0.prev.prev

NormalPlayer('12295352')

In [10]:
Pl1 = player("11744537")

In [11]:
Pl1.config

{'prevId': '11736871', 'prevdeadmatchId': 3894205, 'prevreason': 'skeletonized'}

In [ ]:
        
topSPPs = {}

        
expectedSPPs = {}
for Mu in G.matchups:
    for playerId, p in Mu.config["player"].items():
        Pl = player(str(playerId))
        prevspp0 = 0
        if Pl.prev:
            prevPl = Pl.prev.
        prespp = p.get("prespp", prevspp0)
        spp = p.get("spp", 0)